# Homework 4

In this homework, you will writing two functions that will assign a final letter grade to each student.  The scores of each student on each assignment are given in "Final_Grades.csv". In this semester, there 6 homeworks (columns labeled HW_1, .., HW_6), 3 participation scores (column Part_1, Part_2, Part_3) and a midterm and final.

## Problem 1
Your first task is to write a function callled "Get_Final_Perc" that has the following three inputs:

- HW_weight: The weight towards the final grade of the score on all homeworks.
- Exam_weight: The weight towards the final grade of the midterm and final. You may exam the midterm and final are weighted equally
- Part_weight: The weight towards the final grade of the score on all participation scores.

IMPORTANT:  For the HW and participation categories, the final percentage is (# points earned)/(# of points possible).  So, for example, if there were two HWs and you got 10/10 and 7/8, your final percentage on HW would be 17/18.

Inside of this function, you should

- Read in the data set into a dataframe called df_grades.
- Add to df_grades a column labeled "Final_perc", which gives the final percentage (this should be between 0 and 100 for each student) achieved by each student given the specified weighting scheme that is given as input.  Note that it may be helpful to add other columns along the way, this is okay.


A few other things that you must keep in mind when writing this function

- You should only consider students who have a score for all assignments (you can delete the rows corresponding to students with missing scores for any assignment). Hint: Recall that blanks in a csv are automatically read in as NAs.
- You may assume that for each assignment, there was at least one student who received 100% on the assignment.  Furthermore, there was no extra credit on any assignment. Hint: this is how you can figure out the max score on each assignment.

 


In [128]:
#Run this cell for your imports
import pandas as pd
import numpy as np

def Get_Final_Perc(HW_weight, Exam_weight, Part_weight):

    hw_cols = ['HW_1','HW_2','HW_3','HW_4','HW_5','HW_6']
    part_cols = ['Part_1','Part_2','Part_3']

    df_grades = pd.read_csv("./Data/Final_Grades.csv")
    
    
    
    df_grades['hw_earned_points'] = df_grades[hw_cols].sum(axis=1,skipna=True, numeric_only=True)
    df_grades['hw_possible_points'] = df_grades[hw_cols].max().sum()
    df_grades['hw_perc'] = df_grades['hw_earned_points'] / df_grades['hw_possible_points']
    
    
    df_grades['part_earned_points'] = df_grades[part_cols].sum(axis=1,skipna=True, numeric_only=True)
    df_grades['part_possible_points'] = df_grades[part_cols].max().sum()
    df_grades['part_perc'] = df_grades['part_earned_points'] / df_grades['part_possible_points']

    df_grades['mid_exam_perc'] = df_grades['Midterm'] / df_grades['Midterm'].max()
    df_grades['final_exam_perc'] = df_grades['Final'] / df_grades['Final'].max()

    df_grades['Final_perc'] =  df_grades['hw_perc'] * HW_weight + df_grades['part_perc'] * Part_weight \
        + (df_grades['mid_exam_perc'] + df_grades['final_exam_perc'] ) * Exam_weight

    
    return df_grades

In [129]:
assert np.isclose(Get_Final_Perc(40, 25, 10).Final_perc.mean(),88.711)

In [130]:
assert np.isclose(Get_Final_Perc(100, 0, 0).Final_perc.mean(),97.426)

In [131]:
assert np.isclose(Get_Final_Perc(1, 49, 1).Final_perc.mean(),81.4978754798834)

## Problem 2
Your second and final task is to write a function called "Assign_Grades", which as the following two inputs:

- df_grades_w_final_perc: This is the df_grades dataframe with the Final_Perc column added to it.  In other words, you may assume that df_grades_w_final_perc has a column called Final_perc which is each student's final percentage.

- cutoffs: This is a list that always as 4 entries.  The first entry is the minimum score needed for an "A", the second entry is the minumum score needed for a "B", the third entry is the minumum score needed for a "C" and the last entry is the miminum score need for a "D". Any score below this fourth entry is awarded an "F".  You may assume that all cutoffs are unique so you don't have to worry about ties. Finally, you may assume that the cutoffs for better grades are always higher than the cutoffs for lower grades.

You goal is to add to df_grades_w_final_perc a column called "letter", which contains each student's letter grade (as a string, i.e. an A should added as "A") according to the inputted cutoff points.

Hint: The pandas method "cut" could be very helpful here.

In [132]:
def Assign_Grades(df_grades_w_final_perc, cutoffs):
    min_score = -10000
    max_score = 10000
    bins = [min_score] + sorted(cutoffs) + [max_score]
    labels=['F','D','C','B','A']
    df_grades_w_final_perc['letter'] = pd.cut(df_grades_w_final_perc['Final_perc'],bins,labels=labels, right=False)
    
    return df_grades_w_final_perc.letter.value_counts()

In [133]:
df_grades = Get_Final_Perc(40, 25, 10)
cutoffs = [90,80,70,60]
assert Assign_Grades(df_grades, cutoffs)["A"]==49


In [134]:
df_grades = Get_Final_Perc(100, 0, 0)
cutoffs = [100,80,70,60]
assert Assign_Grades(df_grades, cutoffs)["A"]==27